In [1]:
#import libraries
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import numpy as np
import nltk
import re
import datasets
from transformers import BertTokenizerFast
import transformers
import optuna
import json
import torch
import random
from transformers import BertForSequenceClassification
from sklearn.decomposition import PCA

In [2]:
#read in data
df = pd.read_csv('cleaned_data.csv')
df

,Unnamed: 0,ind,title,text,label,tok_nums
0,0,0,thegoodsoldier,"leonora, however, was not in the least keen on...",noble_agented,"[101, 6506, 6525, 1010, 2174, 1010, 2001, 2025..."
1,1,1,thegoodsoldier,"""i said a great deal more to him than i wante...",guilt_penance_self_loathing,"[101, 1000, 1045, 2056, 1037, 2307, 3066, 2062..."
2,2,2,hamlet,"one woe doth tread upon another's heel, so fas...",senseless_meaning_making,"[101, 2028, 24185, 2063, 11089, 2232, 29449, 2..."
3,3,3,hamlet,"how can that be, unless she drowned herself in...",noble_agented,"[101, 2129, 2064, 2008, 2022, 1010, 4983, 2016..."
4,4,4,hamlet,"if the man go to this water and drown himself,...",noble_agented,"[101, 2065, 1996, 2158, 2175, 2000, 2023, 2300..."
...,...,...,...,...,...,...
255,255,255,thewish,"\n""Thus I sat, as her murderess, at her bedsid...",guilt_penance_self_loathing,"[101, 1000, 2947, 1045, 2938, 1010, 2004, 2014..."
256,256,256,thewish,"Mother, I ask you once more: why did she die?""...",cowardice_foolish_unpleasant,"[101, 2388, 1010, 1045, 3198, 2017, 2320, 2062..."
257,257,257,thewish,"""'But I know myself,' said she. 'I break it of...",guilt_penance_self_loathing,"[101, 1000, 1005, 2021, 1045, 2113, 2870, 1010..."
258,258,258,thewish,"""What is the good of clinging to happiness whe...",guilt_penance_self_loathing,"[101, 1000, 2054, 2003, 1996, 2204, 1997, 1789..."


In [3]:
#create dict to key label strings to numbers
labels = ["noble_agented", "guilt_penance_self_loathing", "senseless_meaning_making", "romantic_reunion", "hopeless_empty_agony", "cowardice_foolish_unpleasant"]
vals = range(0, len(labels))
labels_dict = {k:int(v) for k,v in zip(labels,vals)}
#initialize empty col for labels as numbers
df["lab_nums"] = np.zeros(len(df["ind"]))


In [4]:
#fill empty lab_nums col
for i,label in enumerate(df["label"]):
    df["lab_nums"][i] = int(labels_dict[label])
    
df

<ipython-input-4-dfb1a9247153>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lab_nums"][i] = int(labels_dict[label])


,Unnamed: 0,ind,title,text,label,tok_nums,lab_nums
0,0,0,thegoodsoldier,"leonora, however, was not in the least keen on...",noble_agented,"[101, 6506, 6525, 1010, 2174, 1010, 2001, 2025...",0.0
1,1,1,thegoodsoldier,"""i said a great deal more to him than i wante...",guilt_penance_self_loathing,"[101, 1000, 1045, 2056, 1037, 2307, 3066, 2062...",1.0
2,2,2,hamlet,"one woe doth tread upon another's heel, so fas...",senseless_meaning_making,"[101, 2028, 24185, 2063, 11089, 2232, 29449, 2...",2.0
3,3,3,hamlet,"how can that be, unless she drowned herself in...",noble_agented,"[101, 2129, 2064, 2008, 2022, 1010, 4983, 2016...",0.0
4,4,4,hamlet,"if the man go to this water and drown himself,...",noble_agented,"[101, 2065, 1996, 2158, 2175, 2000, 2023, 2300...",0.0
...,...,...,...,...,...,...,...
255,255,255,thewish,"\n""Thus I sat, as her murderess, at her bedsid...",guilt_penance_self_loathing,"[101, 1000, 2947, 1045, 2938, 1010, 2004, 2014...",1.0
256,256,256,thewish,"Mother, I ask you once more: why did she die?""...",cowardice_foolish_unpleasant,"[101, 2388, 1010, 1045, 3198, 2017, 2320, 2062...",5.0
257,257,257,thewish,"""'But I know myself,' said she. 'I break it of...",guilt_penance_self_loathing,"[101, 1000, 1005, 2021, 1045, 2113, 2870, 1010...",1.0
258,258,258,thewish,"""What is the good of clinging to happiness whe...",guilt_penance_self_loathing,"[101, 1000, 2054, 2003, 1996, 2204, 1997, 1789...",1.0


In [5]:
#subset data to what's necessary for training
data_df = pd.DataFrame(columns=["sentence", "label"])    
data_df["sentence"] = df["text"]
data_df["label"] = df["lab_nums"]

#save to csv as checkpoint
data_df.to_csv('kfold_dataset.csv')     
data_df

,sentence,label
0,"leonora, however, was not in the least keen on...",0.0
1,"""i said a great deal more to him than i wante...",1.0
2,"one woe doth tread upon another's heel, so fas...",2.0
3,"how can that be, unless she drowned herself in...",0.0
4,"if the man go to this water and drown himself,...",0.0
...,...,...
255,"\n""Thus I sat, as her murderess, at her bedsid...",1.0
256,"Mother, I ask you once more: why did she die?""...",5.0
257,"""'But I know myself,' said she. 'I break it of...",1.0
258,"""What is the good of clinging to happiness whe...",1.0


In [6]:
#create 10 train/dev splits
vals_ds = datasets.load_dataset('csv', data_files='kfold_dataset.csv',split=[
    f'train[{k}%:{k+10}%]' for k in range(0, 100, 10)
])
trains_ds = datasets.load_dataset('csv', data_files='kfold_dataset.csv',split=[
    f'train[:{k}%]+train[{k+10}%:]' for k in range(0, 100, 10)
])

Using custom data configuration default-c09c15a55d837c45


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /Users/umadwivedi/.cache/huggingface/datasets/csv/default-c09c15a55d837c45/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/10 [00:00<?, ?it/s]

Using custom data configuration default-c09c15a55d837c45
Reusing dataset csv (/Users/umadwivedi/.cache/huggingface/datasets/csv/default-c09c15a55d837c45/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
#set hyper-hyperparameters
model_name = "bert-base-uncased"
max_length = 3000
#load tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [8]:
#format tokenized training texts in dict
train_texts = {}
for i in range(len(trains_ds)):
    texts=[]
    for j in range(len(trains_ds[i])):
        d = tokenizer(trains_ds[i][j]['sentence'])
        label = np.zeros(6)
        label[int(trains_ds[i][j]['label'])] = 1
        scal = int(trains_ds[i][j]['label'])
        d['label'] = scal
        texts.append(d)
    train_texts[str(i)] = texts 

In [9]:
#format tokenized validation texts in dict
val_texts = {}
for i in range(len(vals_ds)):
    texts=[]
    for j in range(len(vals_ds[i])):
        d = tokenizer(vals_ds[i][j]['sentence'])
        label = np.zeros(6)
        label[int(vals_ds[i][j]['label'])] = 1
        scal = int(vals_ds[i][j]['label'])
        d['label'] = scal
        texts.append(d)
    val_texts[str(i)] = texts 

In [10]:
#compile training and validation in one dict
dat = {"train": train_texts, "validation": val_texts}
#make datasetdict object
dataset = datasets.DatasetDict(dat)

In [11]:
#instantiate model
num_labels = 6
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
#set model training arguments
model_name = model_name.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{'suicide'}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=9,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir='./logs',
    push_to_hub=True,
)

In [13]:
#define function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions[0], axis=1)
    metric = datasets.load_metric('accuracy')
    metric.add_batch(predictions=preds, references=labels)
    acc = metric.compute()
    return acc


In [14]:
#select data split randomly
ind = random.sample(range(len(dataset['train'].keys())), 1)[0]

#instatiate model trainer
trainer = Trainer(
    model,
    args=args,
    train_dataset=dataset["train"][str(ind)],
    eval_dataset=dataset["validation"][str(ind)],
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)
#train model    
trainer.train()

/Users/umadwivedi/Documents/Projects/YALE/senior/s1/LING 380 — neural network language models/final_project/bert-base-uncased-finetuned-suicide is already a clone of https://huggingface.co/psychicautomaton/bert-base-uncased-finetuned-suicide. Make sure you pull the latest changes with `repo.git_pull()`.
***** Running training *****
  Num examples = 234
  Num Epochs = 10
  Instantaneous batch size per device = 9
  Total train batch size (w. parallel, distributed & accumulation) = 9
  Gradient Accumulation steps = 1
  Total optimization steps = 260
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: psychicautomaton (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.783373,0.115385
2,No log,1.569633,0.307692
3,No log,1.545996,0.230769
4,No log,1.658986,0.230769
5,No log,1.428601,0.384615
6,No log,1.442988,0.461538
7,No log,1.417432,0.384615
8,No log,1.287315,0.576923
9,No log,1.312404,0.500000
10,No log,1.334946,0.538462


***** Running Evaluation *****
  Num examples = 26
  Batch size = 2
Saving model checkpoint to bert-base-uncased-finetuned-suicide/checkpoint-26
Configuration saved in bert-base-uncased-finetuned-suicide/checkpoint-26/config.json
Model weights saved in bert-base-uncased-finetuned-suicide/checkpoint-26/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-suicide/checkpoint-26/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-suicide/checkpoint-26/special_tokens_map.json
tokenizer config file saved in bert-base-uncased-finetuned-suicide/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-suicide/special_tokens_map.json
Several commits (3) will be pushed upstream.
***** Running Evaluation *****
  Num examples = 26
  Batch size = 2
Saving model checkpoint to bert-base-uncased-finetuned-suicide/checkpoint-52
Configuration saved in bert-base-uncased-finetuned-suicide/checkpoint-52/config.json
Model weights saved 

TrainOutput(global_step=260, training_loss=1.1855906559870792, metrics={'train_runtime': 2646.1149, 'train_samples_per_second': 0.884, 'train_steps_per_second': 0.098, 'total_flos': 196944156145044.0, 'train_loss': 1.1855906559870792, 'epoch': 10.0})

In [19]:
#look at model structure
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
#load trained model from checkpoint so you don't have to repeat training
#-- note! instructors will not be able to do this
#from the github repo because the bin model file would not upload bc it's too big
#you should be fine with the zipped directory on canvas
output_model_file = 'bert-base-uncased-finetuned-suicide/pytorch_model.bin'
model_state_dict = torch.load(output_model_file) 
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', state_dict=model_state_dict, num_labels=6,output_hidden_states=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/umadwivedi/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layer

In [30]:
#get word to BERT token index dict
with open('tokenizer.json') as json_file:
    vocab = json.load(json_file)['model']['vocab']

In [31]:
#words of interest
suicide_list = ["suicide", 
                "courage", 
                "guilt", 
                "shame", 
                "joy", 
                "ecstasy", 
                "despair", 
                "misery", 
                "romance", 
                "agency", 
                "nobility",
                "meaning",
                "meaningless",
                "meaningful",
                "punishment",
                "hopeless",
                "hope",
                "escape"]

#fill dict with word of interest and its token index in embeddings
suicide_dict = {}

for word in suicide_list:
    suicide_dict[word] = vocab[word]
    


In [34]:
#get pretrained embeddings
gen_embeds = model.get_input_embeddings()
gen_embeds

Embedding(30522, 768, padding_idx=0)

In [45]:
#get learned weights
em_weights = model_state_dict['bert.embeddings.word_embeddings.weight']

In [65]:
#get new embeds
embeds_dict = {}

for i in range(len(suicide_dict.keys())):
    word = list(suicide_dict.keys())[i]
    ind = suicide_dict[word]
    lookup_tensor = torch.tensor([suicide_dict[word]], dtype=torch.long)
    raw = gen_embeds(lookup_tensor).detach().numpy()
    embeds_dict[word] = raw*np.array(em_weights[ind])
    
    

In [67]:
#dict of pretrained embeddings
gen_embeds_dict = {}

for i in range(len(suicide_dict.keys())):
    word = list(suicide_dict.keys())[i]
    lookup_tensor = torch.tensor([suicide_dict[word]], dtype=torch.long)
    gen_embeds_dict[word] = gen_embeds(lookup_tensor).detach().numpy()

In [63]:
#check that embeddings aren't the same
embeds_dict['suicide'] - gen_embeds_dict['suicide']

tensor([[ 3.3231e-02, -1.5140e-02,  7.0230e-02,  1.1353e-02,  9.2188e-02,
          5.0092e-02, -7.4500e-03,  6.2658e-02, -3.3872e-02,  2.0832e-02,
          3.9862e-02,  8.1161e-02,  3.1511e-02,  1.0292e-02, -1.3431e-02,
          3.8536e-02,  2.4391e-02,  1.9971e-02, -3.4887e-02, -2.8261e-02,
          2.8978e-02,  8.6722e-02, -2.3939e-03,  4.9538e-02,  4.7348e-03,
          1.0622e-01,  3.6979e-02, -1.9733e-03,  2.2047e-02,  6.1249e-02,
          1.1682e-02, -1.2333e-02,  3.8076e-03,  2.5259e-02, -2.9316e-02,
          1.5769e-02,  5.0311e-02, -2.1167e-02, -2.2294e-03,  1.3587e-02,
          7.6778e-02,  6.3121e-02, -9.6975e-03,  4.2445e-02, -8.1854e-03,
          4.5742e-02,  4.2900e-03,  4.2281e-02,  2.1551e-03,  2.0152e-02,
         -1.5598e-02, -4.9590e-02,  8.0576e-02, -3.9811e-02, -2.8291e-02,
          1.2898e-01, -2.6500e-02,  2.9023e-02,  2.8383e-02,  9.3874e-02,
          4.4358e-03,  3.8665e-02, -6.3605e-03,  2.8731e-02,  1.3130e-02,
          4.3289e-02, -1.5431e-04,  3.

In [68]:
#dataframe with new embeddings
finetuned = pd.DataFrame(columns=range(len(list(embeds_dict['suicide'][0]))))
for i, word in enumerate(list(suicide_dict.keys())):
    finetuned.loc[i,:] = list(embeds_dict[word][0])

finetuned

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.001036,0.000236,0.004341,0.000126,0.00722,0.002285,0.000056,0.0035,0.001232,0.000417,...,0.002875,0.002039,0.0,0.004198,0.000043,0.001818,0.000055,0.004466,0.00007,0.006869
1,0.001633,0.001055,0.001508,0.001337,0.000224,0.003158,0.000199,0.002035,0.000033,0.004367,...,0.004047,0.007014,0.000645,0.004294,0.010297,0.00025,0.000605,0.000022,0.008244,0.002385
2,0.00062,0.000527,0.000062,0.000748,0.00029,0.002701,0.002219,0.001808,0.005543,0.002104,...,0.002543,0.000172,0.000603,0.005171,0.004048,0.000437,0.005253,0.000389,0.000018,0.000889
3,0.000272,0.004882,0.000017,0.0,0.000323,0.000001,0.001218,0.002926,0.004527,0.000088,...,0.000037,0.000414,0.001603,0.003136,0.002462,0.000046,0.005404,0.000007,0.000706,0.016868
4,0.00229,0.000632,0.000013,0.000527,0.000026,0.00489,0.001976,0.001975,0.002645,0.003032,...,0.001456,0.001198,0.003191,0.0,0.000044,0.000028,0.000529,0.000608,0.000212,0.005936
5,0.000146,0.000053,0.001614,0.001939,0.000302,0.00527,0.002365,0.004933,0.003272,0.000152,...,0.000861,0.000016,0.001026,0.000847,0.000051,0.00005,0.000269,0.000775,0.002969,0.016121
6,0.001562,0.004002,0.00639,0.00008,0.00621,0.00023,0.002496,0.000423,0.001179,0.001187,...,0.000025,0.000456,0.000033,0.000788,0.0,0.000069,0.000263,0.00004,0.000548,0.008951
7,0.000148,0.000937,0.001131,0.002582,0.003639,0.002949,0.003304,0.00396,0.001022,0.004895,...,0.001279,0.002511,0.004448,0.001061,0.001151,0.00009,0.000154,0.001443,0.003953,0.013421
8,0.002309,0.000184,0.001338,0.000095,0.000564,0.000001,0.000121,0.000421,0.001694,0.000045,...,0.004716,0.001223,0.005008,0.001054,0.000001,0.000394,0.000588,0.000122,0.002374,0.002719
9,0.000161,0.000093,0.006557,0.001191,0.000034,0.000192,0.002251,0.000175,0.000608,0.003551,...,0.00142,0.000636,0.00042,0.003722,0.000087,0.000044,0.000126,0.000436,0.000585,0.002599


In [69]:
#dataframe with old embeddings
generic = pd.DataFrame(columns=range(len(list(gen_embeds_dict['suicide'][0]))))
for i, word in enumerate(list(suicide_dict.keys())):
    generic.loc[i,:] = list(gen_embeds_dict[word][0])

generic

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.032195,0.015377,-0.065888,-0.011227,-0.084968,-0.047807,0.007506,-0.059158,0.035104,-0.020415,...,0.053621,-0.045152,-0.000582,-0.064795,0.006588,0.042643,-0.007404,0.066831,0.00837,-0.082878
1,-0.040416,-0.032474,-0.038835,-0.036569,0.014961,-0.056198,-0.014118,-0.045113,-0.005729,-0.066084,...,-0.063616,-0.083751,0.0254,-0.06553,-0.101476,-0.015825,-0.024587,-0.004644,-0.090796,-0.048837
2,-0.024893,-0.022964,0.0079,-0.027356,-0.017038,-0.051974,-0.047102,-0.042519,-0.074454,-0.045871,...,-0.050427,-0.013103,-0.024566,-0.071907,-0.063625,-0.020901,-0.072481,0.019714,-0.004297,-0.029812
3,-0.016499,-0.069873,0.004089,-0.000212,-0.017976,0.00105,-0.034903,-0.054091,-0.067285,0.009372,...,0.006077,-0.020341,-0.040036,-0.056004,-0.04962,0.006775,-0.073514,0.002619,-0.026571,-0.129875
4,-0.047859,-0.025147,-0.003667,0.022963,0.005074,-0.06993,-0.044456,-0.044442,-0.051428,-0.055066,...,-0.038152,-0.034611,-0.05649,0.000347,-0.006596,0.005326,-0.023009,-0.024648,-0.014543,-0.077043
5,0.01207,-0.007247,-0.040171,-0.044033,-0.017387,-0.072592,-0.048629,-0.070234,-0.057202,-0.012341,...,-0.029349,-0.003988,-0.032026,-0.029099,-0.007122,0.007039,0.016412,-0.027831,-0.054487,-0.126967
6,-0.039521,-0.063264,-0.079941,0.008971,0.078805,-0.015157,-0.049956,-0.020557,-0.034338,-0.034446,...,-0.004999,-0.021351,0.005732,-0.028065,0.000051,-0.008309,0.016225,-0.00635,-0.023418,-0.094609
7,0.012147,-0.030616,-0.033624,-0.050812,0.060321,-0.054309,-0.057479,-0.062931,-0.031971,-0.069967,...,-0.035764,-0.050114,0.066696,-0.032573,-0.033934,-0.009482,-0.01241,0.037991,-0.062871,-0.115848
8,-0.048053,0.013559,-0.036582,-0.009722,0.023755,-0.001023,-0.010981,-0.020512,0.041153,-0.006717,...,-0.068673,-0.03497,-0.070767,-0.032472,0.000973,0.019857,-0.024249,0.011052,-0.048721,-0.052141
9,0.012672,-0.009662,-0.080972,-0.034515,-0.005849,0.013862,0.047449,0.013215,0.02466,-0.059589,...,-0.037685,-0.025229,-0.020488,-0.06101,-0.009308,-0.006612,-0.011241,-0.02089,0.02419,-0.050977


In [81]:
#run pca on finetuned embeddings
pca = PCA(n_components=2)
f_principalComponents = pca.fit_transform(finetuned)
f_principalDf = pd.DataFrame(data = f_principalComponents, columns = ['pc1', 'pc2'])
f_principalDf["word"] = list(suicide_dict.keys())
f_principalDf.to_csv('finetuned.csv')
f_principalDf

,pc1,pc2,word
0,-0.011878,-0.004737,suicide
1,-0.022647,0.015876,courage
2,-0.014738,0.006162,guilt
3,-0.015615,0.002057,shame
4,-0.000131,-0.003212,joy
5,0.024431,-0.037805,ecstasy
6,0.002295,-0.022583,despair
7,0.005079,-0.033537,misery
8,-0.021701,0.002311,romance
9,-0.019892,0.018024,agency


In [80]:
#run pca on old embeddings
g_principalComponents = pca.fit_transform(generic)
g_principalDf = pd.DataFrame(data = g_principalComponents, columns = ['pc1', 'pc2'])
g_principalDf['word'] = list(suicide_dict.keys())
g_principalDf

g_principalDf.to_csv('generic.csv')

g_principalDf

,pc1,pc2,word
0,-0.014640,-0.442381,suicide
1,-0.250013,-0.040207,courage
2,-0.034430,-0.124596,guilt
3,-0.001847,-0.124881,shame
4,-0.183386,0.027150,joy
5,0.233327,-0.127098,ecstasy
6,0.167299,-0.216722,despair
7,0.237286,-0.101692,misery
8,-0.142678,-0.156663,romance
9,-0.319359,0.204361,agency


In [ ]:
#empty dataframes for cosine similarity matrices
gen_sim = pd.DataFrame(columns=list(suicide_dict.keys()), index=list(suicide_dict.keys()))
fin_sim = pd.DataFrame(columns=list(suicide_dict.keys()), index=list(suicide_dict.keys()))

In [ ]:
#fill matrices w cosine similarity
for i, word in enumerate(list(suicide_dict.keys())):
    for j, w in enumerate(list(suicide_dict.keys())):
        gen_sim.iloc[i,j] = np.dot(generic.iloc[i,:], generic.iloc[j,:])/(np.linalg.norm(generic.iloc[i,:])*np.linalg.norm(generic.iloc[j,:]))
        fin_sim.iloc[i,j] = np.dot(finetuned.iloc[i,:], finetuned.iloc[j,:])/(np.linalg.norm(finetuned.iloc[i,:])*np.linalg.norm(finetuned.iloc[j,:]))

    
gen_sim.to_csv('gen_sim.csv')
fin_sim.to_csv('fin_sim.csv')